# Anàlisi de rendiment de sucursals i productes

In [8]:
# Ets analista de dades d’una empresa amb diverses sucursals que venen productes de diferents categories.
# El teu objectiu és analitzar el rendiment de vendes i beneficis al llarg dels anys i per sucursal, utilitzant pandas.

In [9]:
# Importa pandas com a pd i numpy com a np. 

In [21]:
import pandas as pd
import numpy as np


In [11]:
# Creació del DataFrame

# Crea un nou DataFrame anomenat sales_df amb les columnes següents:

# branch (sucursal) → "Barcelona", "Madrid", "València", "Sevilla"

# category → "Tecnologia", "Moda", "Alimentació"

# year → un valor entre 2018 i 2023

# units_sold → nombre d’unitats venudes (enter)

# price_per_unit → preu per unitat (float)

# cost_per_unit → cost de producció o compra (float)

# Genera les dades aleatòriament amb numpy per a unes 100 files.
# Afegeix una columna calculada anomenada profit = (price_per_unit - cost_per_unit) * units_sold.

# Mostra les 5 primeres files.



In [27]:
# Genera dades aleatòries per sales_df (~100 files)
np.random.seed(42)

n = 100
branches = ["Barcelona","Madrid","València","Sevilla"]
categories = ["Tecnologia","Moda","Alimentació"]

sales_df = pd.DataFrame({
    'branch': np.random.choice(branches, n),
    'category': np.random.choice(categories, n),
    'year': np.random.choice(np.arange(2018,2024), n),
    'units_sold': np.random.poisson(lam=50, size=n).clip(min=1),
    'price_per_unit': np.round(np.random.uniform(5,1500, size=n),2),
    'cost_per_unit': np.round(np.random.uniform(1,1200, size=n),2)
})

# Calcula profit
sales_df['profit'] = (sales_df['price_per_unit'] - sales_df['cost_per_unit']) * sales_df['units_sold']
# Calcula línia de vendes: price * units
sales_df['sales_line'] = sales_df['price_per_unit'] * sales_df['units_sold']

# Mostra les 5 primeres files
sales_df.head()

,branch,category,year,units_sold,price_per_unit,cost_per_unit,profit,sales_line
0,València,Alimentació,2021,43,1453.13,830.37,26778.68,62484.59
1,Sevilla,Moda,2021,54,1033.60,641.68,21163.68,55814.40
2,Barcelona,Moda,2021,50,1256.26,900.14,17806.00,62813.00
3,València,Moda,2021,51,1300.97,1095.89,10459.08,66349.47
4,València,Moda,2021,43,1258.53,702.59,23905.42,54116.79


In [13]:
# Estadístiques generals

# Fes servir .describe() per obtenir una visió general de les columnes numèriques.

In [23]:
# Estadístiques generals de columnes numèriques
sales_df.describe()


,year,units_sold,price_per_unit,cost_per_unit,profit
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,2020.230000,49.380000,733.294500,614.773700,6155.675400
std,1.739935,6.608504,432.949966,326.493175,25754.879592
min,2018.000000,32.000000,25.440000,4.110000,-47357.000000
25%,2018.750000,45.000000,381.625000,367.077500,-11844.852500
50%,2020.000000,50.000000,648.260000,641.725000,6356.110000
75%,2021.000000,54.000000,1113.970000,878.997500,23055.435000
max,2023.000000,65.000000,1493.770000,1183.520000,67093.120000


In [15]:
# Agrupació per categoria i any

# Agrupa per category i year i calcula:

# Total de vendes (total_sales = sumatori de price_per_unit * units_sold)

# Benefici mitjà (profit.mean())

# Unitats venudes totals (units_sold.sum())

# Desa-ho en un DataFrame anomenat summary_df.

In [24]:
# Agrupa per category i year i calcula total_sales, profit_mean, units_sold_sum
# total_sales ha de ser la suma de price_per_unit * units_sold dins del grup
summary_df = sales_df.copy()
summary_df['sales_line'] = summary_df['price_per_unit'] * summary_df['units_sold']

summary_df = summary_df.groupby(['category','year']).agg(
    total_sales = ('sales_line','sum'),
    profit_mean = ('profit','mean'),
    units_sold_sum = ('units_sold','sum')
).reset_index()

summary_df

,category,year,total_sales,profit_mean,units_sold_sum
0,Alimentació,2018,221042.36,-4950.454444,435
1,Alimentació,2019,102964.11,19984.986667,161
2,Alimentació,2020,313770.73,4110.662500,409
3,Alimentació,2021,458054.94,-969.894167,587
4,Alimentació,2022,95243.11,27552.825000,113
5,Alimentació,2023,313134.92,29668.264000,268
6,Moda,2018,129143.81,-1101.868000,220
7,Moda,2019,51245.17,-1356.690000,109
8,Moda,2020,40519.50,-6768.190000,147
9,Moda,2021,329386.06,5730.628750,393


In [17]:
# Afegir columna calculada i ordenar

# A summary_df, crea una columna margin_percent = (profit_mean / (total_sales / units_sold_sum)) * 100.
# Ordena el resultat pel marge més alt (margin_percent) descendent.

In [25]:
# Afegeix margin_percent i ordena per marge descendent
# margin_percent = (profit_mean / (total_sales / units_sold_sum)) * 100
summary_df['margin_percent'] = (summary_df['profit_mean'] / (summary_df['total_sales'] / summary_df['units_sold_sum'])) * 100
summary_df = summary_df.sort_values('margin_percent', ascending=False).reset_index(drop=True)

summary_df

,category,year,total_sales,profit_mean,units_sold_sum,margin_percent
0,Alimentació,2022,95243.11,27552.825000,113,3268.970559
1,Alimentació,2019,102964.11,19984.986667,161,3124.955728
2,Alimentació,2023,313134.92,29668.264000,268,2539.191334
3,Tecnologia,2020,417098.80,24662.296250,421,2489.296714
4,Moda,2022,111824.82,25472.295000,98,2232.317396
5,Tecnologia,2019,213815.76,16925.428333,271,2145.207200
6,Tecnologia,2022,86415.15,6146.470000,137,974.443011
7,Moda,2021,329386.06,5730.628750,393,683.737830
8,Tecnologia,2018,396330.33,4631.573636,543,634.557664
9,Alimentació,2020,313770.73,4110.662500,409,535.824665


In [19]:
# Agrupació per sucursal i categoria

# Agrupa per branch i category, i calcula:

# total_sales

# profit_total

# avg_price

# avg_cost

# Desa-ho en branch_summary.

In [28]:
# Agrupa per branch i category i calcula les mètriques sol·licitades
branch_summary = sales_df.groupby(['branch','category']).agg(
    total_sales = ('sales_line','sum'),
    profit_total = ('profit','sum'),
    avg_price = ('price_per_unit','mean'),
    avg_cost = ('cost_per_unit','mean')
).reset_index()

branch_summary

,branch,category,total_sales,profit_total,avg_price,avg_cost
0,Barcelona,Alimentació,224107.58,-42157.07,606.937143,761.407143
1,Barcelona,Moda,147229.80,-59920.62,406.277143,571.470000
2,Barcelona,Tecnologia,267587.55,161151.07,886.466667,334.205000
3,Madrid,Alimentació,298247.01,-33010.28,617.390000,646.768000
4,Madrid,Moda,172502.05,2254.54,530.662857,554.900000
5,Madrid,Tecnologia,247967.19,-72462.11,577.927778,747.501111
6,Sevilla,Alimentació,494416.22,57518.22,818.400000,725.415000
7,Sevilla,Moda,206107.63,18968.65,787.896000,713.568000
8,Sevilla,Tecnologia,517484.77,131669.35,797.283077,611.602308
9,València,Alimentació,487439.36,257743.54,945.475000,460.125000


In [ ]:
# Filtratge de resultats

# Mostra només les sucursals on el profit_total superi la mitjana de tots els beneficis totals.

In [30]:
# Mostra només les sucursals on el profit_total superi la mitjana de tots els profit_total
mean_profit_total = branch_summary['profit_total'].mean()
top_branches = branch_summary[branch_summary['profit_total'] > mean_profit_total]

top_branches

,branch,category,total_sales,profit_total,avg_price,avg_cost
2,Barcelona,Tecnologia,267587.55,161151.07,886.466667,334.205000
6,Sevilla,Alimentació,494416.22,57518.22,818.400000,725.415000
8,Sevilla,Tecnologia,517484.77,131669.35,797.283077,611.602308
9,València,Alimentació,487439.36,257743.54,945.475000,460.125000
10,València,Moda,321886.10,106060.33,982.420000,656.210000
11,València,Tecnologia,260217.40,87751.92,767.372857,520.591429


In [ ]:
# Transformació de dades

# Afegeix una nova columna a sales_df que representi el benefici normalitzat per categoria, és a dir, cada valor de profit dividit pel màxim profit dins la seva categoria.

In [31]:
# Normalitza el benefici dins de cada categoria: profit_normalized = profit / max(profit) per categoria
sales_df['profit_normalized'] = sales_df.groupby('category')['profit'].transform(lambda x: x / x.max() if x.max() != 0 else 0)

sales_df[['category','product' if 'product' in sales_df.columns else 'branch','profit','profit_normalized']].head()

,category,branch,profit,profit_normalized
0,Alimentació,València,26778.68,0.399127
1,Moda,Sevilla,21163.68,0.466770
2,Moda,Barcelona,17806.00,0.392716
3,Moda,València,10459.08,0.230678
4,Moda,València,23905.42,0.527240


In [ ]:
# Filtrat amb funció personalitzada

# Defineix una funció filter_func(x) que retorni només els grups (per categoria) on el desviament estàndard del benefici (profit.std()) sigui superior a 5000.
# Aplica-la amb .groupby('category').filter(filter_func).

In [32]:
# Filtra grups per categoria on la desviació estàndard del profit és superior a 5000

def filter_func(x):
    return x['profit'].std() > 5000

filtered_groups = sales_df.groupby('category').filter(filter_func)

filtered_groups.head()

,branch,category,year,units_sold,price_per_unit,cost_per_unit,profit,sales_line,profit_normalized
0,València,Alimentació,2021,43,1453.13,830.37,26778.68,62484.59,0.399127
1,Sevilla,Moda,2021,54,1033.60,641.68,21163.68,55814.40,0.466770
2,Barcelona,Moda,2021,50,1256.26,900.14,17806.00,62813.00,0.392716
3,València,Moda,2021,51,1300.97,1095.89,10459.08,66349.47,0.230678
4,València,Moda,2021,43,1258.53,702.59,23905.42,54116.79,0.527240


In [ ]:
# Anàlisi addicional per dècada

# Crea una nova columna decade agrupant els anys en dècades (2010s, 2020s, etc.).
# Agrupa per decade i category i mostra el nombre total d’unitats venudes i el benefici mitjà.

In [33]:
# Crea columna decade i agrupa per decade i category
sales_df['decade'] = (sales_df['year'] // 10) * 10

decade_summary = sales_df.groupby(['decade','category']).agg(
    units_sold_total = ('units_sold','sum'),
    profit_mean = ('profit','mean')
).reset_index()

decade_summary

,decade,category,units_sold_total,profit_mean
0,2010,Alimentació,596,1283.405833
1,2010,Moda,329,-1174.674286
2,2010,Tecnologia,814,8970.581176
3,2020,Alimentació,1377,8321.982963
4,2020,Moda,937,3978.190526
5,2020,Tecnologia,885,8645.019444
